In [1]:
!pip install numpy
!pip install sanssouci
!pip install unittest

ERROR: Could not find a version that satisfies the requirement unittest (from versions: none)
ERROR: No matching distribution found for unittest


In [2]:
import numpy as np
import matplotlib.pyplot as plt
import unittest
from sanssouci import curve_max_fp, find_largest_region

In [3]:
def interpolation_naif(p_values: list, thresholds: list, zeta: list=[k for k in range(5000)]) -> int:
    "Naive calculus posthoc bounds"
    p_values = np.sort(p_values)
    K = len(thresholds)
    kmax = min(len(p_values), K)
    B = []
    for k in range(kmax):
        compteur = 0
        for p_value in p_values:
            if p_value >= thresholds[k]:
                compteur += 1
        B.append(compteur + zeta[k])
    if min(B) <= len(p_values):
        print("La borne correspond au k :", np.argmin(B))
    else:
        print("Nous avons la borne triviale")
    return min(min(B), len(p_values))

In [4]:
def interpolation(p_values: list, thresholds: list, zeta: list=[k for k in range(5000)]) -> int:
    "Calcul posthoc bounds"
    p_values = np.sort(p_values)
    K = len(thresholds)
    kmax = min(len(p_values), K)
    B = []
    i_start = 0
    for k in range(kmax):
        i = i_start
        while i < len(p_values) and p_values[i] < thresholds[k]:
            i += 1
            i_start = i
        B.append(len(p_values) - i + zeta[k]) # k est un cas particulier de Zeta_k
    # if min(B) <= len(p_values):
    #     # print("La borne correspond au k", np.argmin(B))
    # else:
    #     print("Nous avons la borne triviale")
    # return min(min(B), len(p_values))
    return np.argmin(B)

Quelle est la complexité de cet algorithme?
Trier la liste des p-valeurs : O(slog(s))
Parcourir la liste des indices k : O(k)
Parcourir la liste des p-valeurs : O(log(s))
Chercher le minimum dans la liste ?

In [5]:
def interpolation_bis(p_values: list, thresholds:list, zeta: list=[k for k in range(5000)]) -> int:
    p_values = np.sort(p_values)
    s = len(p_values)
    K = len(thresholds)
    B = []
    k, i = 0, 0
    while k < K and i < s:
        if p_values[i] < thresholds[k]:
            i += 1
        else:
            B.append(s - i + zeta[k])
            k += 1
    B.append(s - i + zeta[k])
    if min(B) < len(p_values):
            print("La borne correspond au k :", np.argmin(B))
    else:
        print("Nous avons la borne triviale")
    return min(min(B), len(p_values))


In [6]:
def interpolation_minmax_naif(p_values: list, thresholds: list, kmin:int, kmax: int, zeta: list=[k for k in range(5000)]) -> int:
    "Naive calculus of the posthoc bounds with kmin and kmax"
    p_values = np.sort(p_values)
    B = []
    if kmin == kmax:
        return len(p_values)
    else:
        for k in range(kmin, kmax):
            compteur = 0
            for p_value in p_values:
                if p_value >= thresholds[k]:
                    compteur += 1
            B.append(compteur + zeta[k])
        if min(B) <= len(p_values):
            print("La borne correspond au k :", np.argmin(B))
        else:
            print("Nous avons la borne triviale")
        return min(min(B), len(p_values))

In [7]:
def interpolation_minmax(p_values: list, thresholds: list, kmin: int, kmax: int, zeta: list= [k for k in range(5000)]) -> int:
    "La bound posthoc est calculée pour les k compris entre kmin et kmax"
    p_values = np.sort(p_values)
    B = []
    i_start = 0
    if kmin == kmax:
        i = 0
        while i < len(p_values):
            if p_values[i] < thresholds[kmax - 1]:
                i += 1
            else:
                return min(len(p_values) - i + zeta[kmax - 1], len(p_values))
    else:
        for k in range(kmin, min(kmax, len(p_values))):
            i = i_start
            while i < len(p_values) and p_values[i] < thresholds[k]:
                    i += 1
                    i_start = i
            B.append(len(p_values) - i + zeta[k]) # k est un cas particulier de Zeta_k
        if min(B) < len(p_values):
            print("La borne correspond au k :", np.argmin(B))
        else:
            print("Nous avons la borne triviale")
        return min(min(B), len(p_values))

In [8]:
m = 1000
alpha = 0.9
K = 500

np.random.seed(23)
values = np.linspace(0, 1, 300)
# p_values = values ** 2
p_values = [np.random.beta(a=k, b=m-k+1) for k in range(1, K)]
thresholds = np.array([alpha*k/m for k in range(K)])

In [9]:
interpolation_minmax(p_values, thresholds, kmin=0, kmax=K)

La borne correspond au k : 2


497

In [10]:
def linear_interpolation(p_values : list, thresholds: list, kmin: int=0)-> list:
    p_values = np.sort(p_values)
    s = len(p_values)
    K = len(thresholds)
    tau = np.zeros(s)
    for k in range(s):
        if k < kmin:
            tau[k] = 0
        elif kmin <= k < K:
            tau[k] = thresholds[k]
        else:
            tau[k] = thresholds[K-1]
    kappa = np.ones(s, dtype=int)*s
    r = np.ones(s, dtype=int)*s
    k, i = 0, 0
    while k < s and i < s:
        if p_values[i] < tau[k]:
            kappa[i] = k
            i += 1
        else:
            r[k] = i
            k += 1
    V, A, M = np.zeros(s), np.zeros(s), np.zeros(s)
    M[0] = r[0]
    for k in range(s):
        A[k] = r[k] - k
        if k > 0:
            M[k] = max(M[k-1], A[k])      
    for i in range(s):
        if kappa[i] > 0:
            V[i] = int(min(kappa[i], i+1 - M[kappa[i]-1]))
    return v

In [11]:
def interpolation_segments(p_values: list, thresholds: list, L: list)-> int:
    "La bound posthoc est calculée pour les k entre les (kmin, kmax)"
    return min([interpolation_minmax(p_values, thresholds, kmin= element[0], kmax=element[1]) for element in L])

In [12]:
def linear_interpolation_zeta(p_values: list, thresholds: list, zeta: list, kmin: int=0) -> list:
    p_values = np.sort(p_values)
    s = len(p_values)
    K = len(thresholds)
    tau = np.zeros(K)
    ksi = np.zeros(K)
    K = max(s, K)
    for k in range(K):
        if k < kmin:
            tau[k] = 0
            ksi[k] = zeta[k]
        elif kmin <= k < len(thresholds):
            tau[k] = thresholds[k]
            ksi[k] = zeta[k]
        else:
            tau[k] = thresholds[len(thresholds)-1]
            ksi[k] = zeta[len(thresholds)-1]
    kappa = np.ones(K, dtype=int)*K
    r = np.ones(K, dtype=int)*s
    k, i = 0, 0
    while k < K and i < s:
        if p_values[i] < tau[k]:
            kappa[i] = k
            i += 1
        else:
            r[k] = i
            k += 1
    V, A, M = np.zeros(s), np.zeros(K), np.zeros(K)
    M[0] = r[0]
    for k in range(K):
        A[k] = r[k] - ksi[k]
        if k > 0:
            M[k] = max(M[k-1], A[k])
    for i in range(s):
        if kappa[i] > 0:
            V[i] = int(min(ksi[kappa[i]], i+1 - M[kappa[i]-1]))
    return V

In [14]:
m = 10000
alpha = 0.9
K = 600
s = 800
np.random.seed(42)
values = np.linspace(0, 1, s)
# p_values = values ** 2
p_values = [0]*s
# p_values = [np.random.beta(a=k, b=m-k+1) for k in range(1, s+1)]
thresholds = np.array([alpha*k/m for k in range(K)])
# thresholds = np.sort(np.random.uniform(low=0, high=1, size=K))
zeta = [k for k in range(K)]

In [15]:
linear_interpolation_zeta(p_values, thresholds, zeta=zeta)

IndexError: index 600 is out of bounds for axis 0 with size 600